# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта **«Статистический анализ данных»**). Предобработка данных не понадобится — мы её уже сделали.

**Задача** - построить модель для задачи классификации, которая выберет подходящий тариф. 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')
from sklearn.dummy import DummyClassifier

## 1. Общая информация

In [2]:
data = pd.read_csv('C:/Users/leoci/Яндекс Практикум/Projects/Introduction_to_the_machine_learning/users_behavior.csv')
display(data.head(5))
display(data.info())
print(data.duplicated().sum())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

0


Датасет содержит следующие данные за месяц о пользователях:
1. количество звонков
2. количество потраченных минут
3. количество отправленных сообщений
4. количество потраченных мегабайт интернета
5. тариф

В датасете  3214 строк. Пропусков нет. Дубликатов нет. Предобработка данных не требуется

## 2. Разбиение данных на выборки

Данные необходимо разбить на 3 выборки: обучающую, валидационную, тестовую. Разбиение выполним в соотношении 3:1:1. Для разбиения используем функцию train_test_split. Данная функция делит датасет на два, поэтому будем делить в 2 этапа. В первый этап выделим тестовую выборку размером 20% от общих данных, во второй этап оставшиеся данные разделим на обучающую и валидационную выборки, задав параметр test_size как 0.25, потому что нам надо получить 20% изначальных данных от 80% оставшихся данных после первого деления, что составляет одну четверть.

In [3]:
data_pre_train, data_test = train_test_split(data, test_size=0.2, random_state=123)
data_train, data_valid = train_test_split(data_pre_train, test_size=0.25, random_state=123)

print(data_train.shape)
print(data_valid.shape)
print(data_test.shape)

(1928, 5)
(643, 5)
(643, 5)


## 3. Исследование моделей

Целевым признаком в данной задаче является тариф абонента. Разобьем все выборки на целевой признак и остальные признаки.

In [4]:
train_features = data_train.drop('is_ultra', axis=1)
train_target = data_train['is_ultra']

valid_features = data_valid.drop('is_ultra', axis=1)
valid_target = data_valid['is_ultra']

test_features = data_test.drop('is_ultra', axis=1)
test_target = data_test['is_ultra']

Исследуем три модели: дерево решений, случайный лес и логистическую регрессию.

### 3.1. Дерево решений

In [5]:
best_model_tree = None
best_result_tree = 0
best_depth_tree = 0
for depth in range(1, 16):
    model = DecisionTreeClassifier(random_state=123, max_depth=depth)
    model.fit(train_features, train_target)
    result = model.score(valid_features, valid_target)
    if result > best_result_tree:
        best_result_tree = result
        best_model_tree = model
        best_depth_tree = depth
print('')
print('Наилучший результат accuracy = {:.4f}'.format(best_result_tree), 'при глубине дерева', best_depth_tree)


Наилучший результат accuracy = 0.7947 при глубине дерева 9


### 3.2. Случайный лес

In [6]:
best_model_forest = None
best_result_forest = 0
best_depth_forest = 0
best_n_estimators_forest = 0
for est in range(1,16):
    for depth in range(1,11):
        model = RandomForestClassifier(random_state=123, n_estimators=est, max_depth=depth)
        model.fit(train_features, train_target)
        result = model.score(valid_features, valid_target)
        if result > best_result_forest:
            best_result_forest = result
            best_depth_forest = depth
            best_n_estimators_forest = est
            best_model_forest = model
print('')
print('Наилучший результат accuracy = {:.4f}'.format(best_result_forest), 'при количестве деревьев', best_n_estimators_forest, 'и при глубине дерева', best_depth_forest)


Наилучший результат accuracy = 0.8103 при количестве деревьев 8 и при глубине дерева 8


### 3.3. Логистическая регрессия

In [7]:
model = LogisticRegression(random_state=123)
model.fit(train_features, train_target)
result = model.score(valid_features, valid_target)
print('Accuracy = {:.4f}'.format(result))        

Accuracy = 0.7341


### 3.4. Вывод

Наилучшее значение Accuracy получается при использовании модели Случайный лес. Однако, на валидационной выборке Случайный лес не сильно выигрывает в качестве у Дерева решений (всего 1,5% правильных ответов), а Дерево решений значительно быстрее по скорости работы. Качество логистической регрессии сильно уступает другим моделям.

На тестовой выборке проверим обе модели (дерево решений и случайный лес)

## 4. Проверка моделей на тестовой выборке

In [8]:
print('Accuracy модели дерево решений на тестовой выборке: {:.4f}'.format(best_model_tree.score(test_features, test_target)))
print('Accuracy модели случайный лес на тестовой выборке: {:.4f}'.format(best_model_forest.score(test_features, test_target)))

Accuracy модели дерево решений на тестовой выборке: 0.7869
Accuracy модели случайный лес на тестовой выборке: 0.7947


### 4.1. Вывод

На тестовой выборке случайный лес остался лучше чем дерево решений по качеству, однако его превосходство меньше чем 1%, а скорость работы у дерева решений выше. 

Таким образом, может быть целесообразно применять модель дерево решений для прогнозирования больших объемов данных.

## 5. Проверка модели на адекватность

Сравним точность полученных моделей с константной моделью. В константной модели будем всегда предсказывать 0, т.к. пользователей тарифа Смарт больше.

In [9]:
const_model = DummyClassifier(strategy='constant', constant=0)
const_model.fit(test_features, test_target)
print('Точность константной модели основанной на большем классе:', const_model.score(test_features, test_target))

Точность константной модели основанной на большем классе: 0.6998444790046656


### 5.1. Вывод

Обученные модели дерева решений и случайного леса превосходят в точности константную модель. Можно сделать вывод, что модели являются адекватными. А модель логистическая регрессия имеет точность почти равную константной модели, поэтому данная модель не подходит для данной задачи.

## 6. Вывод

В данной работе были обучены модели для предсказания подходящего тарифа для абонента. Были исследованы следующие модели:
1. Дерево решений: работает быстро, обладает достаточно высокой accuracy.
2. Случайный лес: работает медленнее, точность сравнима с деревом решений, разница менее 1%.
3. Логистическая регрессия: имеет точность не сильно выше константной модели, поэтому не подходит для данной задачи.